# PSPNet 

## Set up
1. download resnet 50 model. This will be our initial model. The code will improve it by using PSPNet architecture. 
2. unzip Camvid data
3. check python version

In [1]:
!wget -O "resnet50_v2.pth" --no-check-certificate 'https://docs.google.com/uc?export=download&id=1w5pRmLJXvmQQA5PtCbHhZc_uC4o0YbmA'
!mkdir initmodel && mv resnet50_v2.pth initmodel/

--2023-08-07 20:25:22--  https://docs.google.com/uc?export=download&id=1w5pRmLJXvmQQA5PtCbHhZc_uC4o0YbmA
Resolving docs.google.com (docs.google.com)... 2607:f8b0:4002:c09::64, 2607:f8b0:4002:c09::8b, 2607:f8b0:4002:c09::8a, ...
Connecting to docs.google.com (docs.google.com)|2607:f8b0:4002:c09::64|:443... connected.
HTTP request sent, awaiting response... 303 See Other
Location: https://doc-14-c8-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/nm67gdbkm1gipj06ljdqeq111nd1bije/1691465100000/15543419006824780045/*/1w5pRmLJXvmQQA5PtCbHhZc_uC4o0YbmA?e=download&uuid=edf5bc75-a33d-4c90-a661-b416db5916e2 [following]
--2023-08-07 20:25:27--  https://doc-14-c8-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/nm67gdbkm1gipj06ljdqeq111nd1bije/1691465100000/15543419006824780045/*/1w5pRmLJXvmQQA5PtCbHhZc_uC4o0YbmA?e=download&uuid=edf5bc75-a33d-4c90-a661-b416db5916e2
Resolving doc-14-c8-docs.googleusercontent.com (doc-14-c8-docs.googleusercontent.co

In [2]:
!cd Camvid && unzip camvid_semseg11.zip && cd ..

Archive:  camvid_semseg11.zip
   creating: semseg11/
  inflating: semseg11/0016E5_08085_L.png  
  inflating: semseg11/0006R0_f01080_L.png  
  inflating: semseg11/0016E5_00480_L.png  
  inflating: semseg11/0016E5_01350_L.png  
  inflating: semseg11/0016E5_01560_L.png  
  inflating: semseg11/0016E5_01440_L.png  
  inflating: semseg11/0016E5_06150_L.png  
  inflating: semseg11/0016E5_08045_L.png  
  inflating: semseg11/0016E5_00390_L.png  
  inflating: semseg11/0016E5_07680_L.png  
  inflating: semseg11/0016E5_08139_L.png  
  inflating: semseg11/0016E5_08280_L.png  
  inflating: semseg11/0016E5_01110_L.png  
 extracting: semseg11/0016E5_08550_L.png  
  inflating: semseg11/0006R0_f02850_L.png  
  inflating: semseg11/0016E5_05940_L.png  
  inflating: semseg11/0016E5_07230_L.png  
  inflating: semseg11/0016E5_00720_L.png  
  inflating: semseg11/0006R0_f01320_L.png  
  inflating: semseg11/0016E5_08061_L.png  
  inflating: semseg11/0016E5_08019_L.png  
  inflating: semseg11/0016E5_04980_L.png 

In [4]:
!python3 --version

Python 3.8.10


In [5]:
from types import SimpleNamespace

In [6]:
import os
import torch

## Build PSPNet
### 1) Set up the model with ResNet50
The model will have 5 layers which got from pretrained-deep-based ResNet50. Layer0 will be sequential of conv1 -> bn1 -> relu -> conv2 -> bn2 -> relu -> conv3 -> bn3 -> relu -> maxpool.
